# Imports

In [3]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Optional
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
import nltk
import faiss
import torch.nn.functional as F
from torch.utils.data import Dataset

import os
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

import itertools
import time
import warnings

# Chargement des données

In [ ]:
from datasets import load_dataset
questions = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")
text = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

In [5]:
print(questions)
print(text)

DatasetDict({
    test: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})
DatasetDict({
    passages: Dataset({
        features: ['passage', 'id'],
        num_rows: 3200
    })
})


In [15]:
from datasets import DatasetDict

questions_all = questions["test"]

# split = questions_all.train_test_split(test_size=0.2, seed=42)
# questions_train = split["train"]
# temp = split["test"].train_test_split(test_size=0.5, seed=42)
# questions_val = temp["train"]
# questions_test = temp["test"]

# questions_split = DatasetDict({
#     "train": questions_train,
#     "validation": questions_val,
#     "test": questions_test
# })

questions_val = questions_all

questions_split = DatasetDict({
    "validation": questions_val
})

In [16]:
print(questions_split)

DatasetDict({
    validation: Dataset({
        features: ['question', 'answer', 'id'],
        num_rows: 918
    })
})


## Constantes

In [17]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

## Encodeur

### Chargement de l'encodeur

In [ ]:
from transformers import AutoTokenizer, AutoModel
model_name = 'BAAI/bge-base-en-v1.5'
emb_tokenizer = AutoTokenizer.from_pretrained(model_name)
emb_model = AutoModel.from_pretrained(model_name)
emb_model.to(DEVICE)

### Stockage dans FAISS

In [18]:
def encode_sequences(sequences: list, tokenizer, model, device=DEVICE, batch_size=BATCH_SIZE, training=False) -> torch.Tensor:
    model.train() if training else model.eval()
    context = torch.enable_grad() if training else torch.no_grad()

    embeddings = []
    with context:
        for i in range(0, len(sequences), batch_size):
            batch = sequences[i:i + batch_size]

            # Tokenisation des textes avant entrée du modèle
            inputs = tokenizer(
                batch, padding=True, truncation=True, return_tensors="pt"
            ).to(device)

            # Passage dans le modèle encodeur
            output = model(**inputs).last_hidden_state

            # Pooling : moyenne des vecteurs de tous les tokens pour chaque séquence
            pooled = output.mean(dim=1)  # [batch_size, hidden_size]

            embeddings.append(pooled)

    # Concatène tous les embeddings pour obtenir un seul tensor de sortie
    return torch.cat(embeddings, dim=0)

In [19]:
def retrieve_top_k_passages_faiss(questions: list, vector_index: faiss.IndexFlatL2, embedding_model_tokenizer, embedding_model, k: int):
    question_embeddings = encode_sequences(questions, embedding_model_tokenizer, embedding_model)
    question_embeddings = question_embeddings.cpu().numpy()
    faiss.normalize_L2(question_embeddings)
    _, top_k_indices = vector_index.search(question_embeddings, k)
    
    return top_k_indices

In [12]:
passages = text["passages"]["passage"]
passage_embed = encode_sequences(passages, emb_tokenizer, emb_model)
passage_embeddings = passage_embed.cpu().numpy()
faiss.normalize_L2(passage_embeddings)

d = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(d)
index.add(passage_embeddings)

## Generator

### Chargement du générateur

In [ ]:
from huggingface_hub import login
login(token = '')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

gen_tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",           # auto-placement sur GPU si dispo
    torch_dtype=torch.float16,   # allège la mémoire (fp16)
    trust_remote_code=True,       # pour activer les modèles qui utilisent du code perso
)
gen_tokenizer.pad_token = gen_tokenizer.eos_token

### Prompts

In [ ]:
def generate_prompts(questions, index, emb_tokenizer, emb_model, k, passages):
    prompts = []
    faiss_passages = retrieve_top_k_passages_faiss(questions, index, emb_tokenizer, emb_model, k)
    help_instruction = "Give an answer to the question using one word if possible."

    for i, passages_ids in enumerate(faiss_passages):
        # Récupérer les textes directement via les indices
        retrieved_passages = [passages[j] for j in passages_ids]

        # Construire le prompt
        context_string = (
            f"{help_instruction}\n" +
            "\n".join(retrieved_passages) +
            "\n" +
            f"Question: {questions[i]}\n"
        )
        prompts.append(context_string)

    return prompts

In [23]:
questions_val = questions_split["validation"]["question"]
passages = text["passages"]["passage"]

k_values = [1, 2, 3, 4]
prompts_k = {}

for k in k_values:
    prompts_k[k] = generate_prompts(questions_val, index, emb_tokenizer, emb_model, k, passages)

# Affichage exemple
print(prompts_k[3][0])

Give an answer to the question using one word if possible.
Abraham Lincoln (February 12, 1809 â April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that 'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought Abraham Lincoln to the center stage of American politics at

### Génération

In [24]:
def process_batch_prompts(batch_prompts, tokenizer, model, device, max_new_tokens=64):
    formatted_prompts = [
        f"<s>[INST] {prompt.strip()} [/INST]" for prompt in batch_prompts
    ]

    print(formatted_prompts)

    inputs = tokenizer(formatted_prompts, return_tensors="pt", padding=True, truncation=True).to(device)

    # Génération
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

    # Décodage et extraction de la réponse
    responses = [
        tokenizer.decode(output, skip_special_tokens=True).split('[/INST]')[-1].strip()
        for output in outputs
    ]

    print(responses)

    return responses

def chunked_iterable(iterable, size):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, size))
        if not chunk:
            return
        yield chunk

def generate_responses(gen_model, gen_tokenizer, prompts, batch_size = 32) : 
    batch_responses = []
    
    for batch_index, batch_prompts in enumerate(chunked_iterable(prompts, batch_size)):
        print(f"[k={k} | Batch {batch_index}] Started generation...")
        start_time = time.time()
        
        batch_responses.extend(process_batch_prompts(batch_prompts, gen_tokenizer, gen_model, DEVICE))
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[k={k} | Batch {batch_index}] Time for generation = {elapsed_time:.2f} seconds")
    return batch_responses

In [25]:
k_values = [3]
generated_responses = {}
for k in k_values:
    generated_responses[k] = generate_responses(gen_model, gen_tokenizer, prompts_k[k])
    print(f"Done with k = {k}!")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[k=3 | Batch 0] Started generation...
['<s>[INST] Give an answer to the question using one word if possible.\nAbraham Lincoln (February 12, 1809 â\x80\x93 April 15, 1865) was the sixteenth President of the United States, serving from March 4, 1861 until his assassination. As an outspoken opponent of the expansion of slavery in the United States, "[I]n his short autobiography written for the 1860 presidential campaign, Lincoln would describe his protest in the Illinois legislature as one that \'briefly defined his position on the slavery question, and so far as it goes, it was then the same that it is now." This was in reference to the anti-expansion sentiments he had then expressed. Doris Kearns Goodwin, Team of Rivals: The Political Genius of Abraham Lincoln (2005) p. 91.  Holzer pg. 232.  Writing of the Cooper Union  speech, Holzer notes, "Cooper Union proved a unique confluence of political culture, rhetorical opportunity, technological innovation, and human genius, and it brought A

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Yes.', 'Yes.', 'No.', '18', '1832.', 'Paper currency.', 'Bedell', '1776.', 'No.', 'No. Abraham Lincoln was the 16th President of the United States.', 'Yes.', 'Yes.', 'Kentucky.', 'March 4, 1861.', 'John Wilkes Booth.', 'Yes.', 'McClellan', 'To free slaves in Confederate-held territories.', 'Yes.', 'Yes.', 'Yes.', 'Seven.', '1846.', '23 years.', 'The trail used by Lincoln in the William "Duff" Armstrong murder trial was the one that led to the acquittal of the accused.', 'Yes.', 'No.', 'Avogadro.', 'John Adams', '1811.', '1779', 'No.']
[k=3 | Batch 0] Time for generation = 6.36 seconds
[k=3 | Batch 1] Started generation...
["<s>[INST] Give an answer to the question using one word if possible.\nLorenzo Romano Amedeo Carlo Avogadro, Count of Quaregna and Cerreto (August 9, 1776 July 9, 1856) was an Italian savant.  He is most noted for his contributions to the theory of molarity and molecular weight. As a tribute to him, the number of elementary entities (atoms, molecules, ions or other

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

['Yes.', 'Yes.', 'No.', 'Three.', 'Russian Tortoise.', '1. Encyclopedia of Turtles by Pritchard, Pether C H (1979).\n2. Turtles of the World by Pritchard, Pether C H (1979).\n3. Turtles and Crocodiles of Ins', 'Female.', 'Papillae.', 'Neck.', 'No.', 'Yes', 'Yes.', 'Paracryptodira.', 'Yolk.', 'Beaches.', 'Pleurodira.', 'Turtle eggs are covered in sand when they incubate.', 'Reptiles.', 'Humus.', 'All turtles are ectothermic.', 'Yes.', 'Yes.', 'Papillae have various shapes and sizes.', 'Spherical.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.']
[k=3 | Batch 25] Time for generation = 5.43 seconds
[k=3 | Batch 26] Started generation...
["<s>[INST] Give an answer to the question using one word if possible.\nOn August 22, 1848, Grant married Julia Boggs Dent (1826 1902), the daughter of a slave owner. They had four children: Frederick Dent Grant, Ulysses S. Grant, Jr. (Buck), Ellen Wrenshall Grant (Nellie), and Jesse Root Grant.\nPresident Grant with his wife, Julia, and so

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Julia', 'Point Pleasant, Ohio.', 'Republican.', 'It is not clear from the given information what Grant said in response to the question.', 'Yes.', 'Grant thought the Mexican-American War was unjust because it was designed to gain land open to slavery.', 'Ulysses S. Grant.', '1868.', 'Grant', '1856.', 'Grant', 'Yes.', 'The question is asking about the origin of the phrase "Ma, Ma, where\'s my Pa?" and it was not from Pennsylvania.', 'No.', 'Yes.', 'No.', 'No.', 'Yes.', 'Lincoln delivered his second inaugural address.', 'Crisis.', 'No.', 'Yes.', 'No.', 'Spanish', 'San Carlos.', 'Europeans', 'Cantegriles', 'No.', 'Catholic.', 'Yes', 'Yes.', 'Yes.']
[k=3 | Batch 26] Time for generation = 4.25 seconds
[k=3 | Batch 27] Started generation...
["<s>[INST] Give an answer to the question using one word if possible.\nAt 176,214 square kilometres (68,036 square miles) of continental land and 142,199 square kilometres (54,903 sq mi) of jurisdictional waters and small river islands,  Instituto Naci

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['176,214', '88%', 'Military stronghold.', 'Two.', '65%', 'Yes.', 'No.', 'Yes.', 'Plebiscite.', 'Catholicism, Protestantism, Judaism.', 'Uruguay has been competitive in agriculture by focusing on ecological and natural products, and by developing a strong export-oriented agricultural sector.', 'Partido Colorado, Partido Blanco, Frente Amplio.', 'No.', 'Montevideo', 'Meanings.', 'Import/export.', 'Independence.', 'No.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'Yes.', 'No.', 'Yes.', 'No.', 'Paris.', 'December 28, 1856.', 'Wilson was President when Franklin D. Roosevelt was elected.', 'History and political science.']
[k=3 | Batch 27] Time for generation = 4.01 seconds
[k=3 | Batch 28] Started generation...
['<s>[INST] Give an answer to the question using one word if possible.\nWilson spent 1914 through the beginning of 1917 trying to keep America out of the war in Europe. He offered to be a mediator, but neither the Allies nor the Central Powers took his requests seriously. Repub

## Evaluation

In [28]:
questions_val = questions_split["validation"].to_pandas()
df_export = questions_val[['id', 'question', 'answer']].copy()
df_export['generated_answer'] = generated_responses[k]
df_export.to_csv(f"generated_responses_simple.csv", index=False, encoding='utf-8')